# Opportunity Score
This score is based on 8 different categories. 

1. Demographics: Understanding the target customer base and selecting a location with a relevant demographic profile.
2. Population Density: Selecting a location with a sufficient population density to support the customer base. 
3. Foot Traffic: Opting for a busy area with high pedestrian traffic to attract potential customers. 
4. Competition: Assessing the level of competition in the vicinity and choosing a location with a balanced market. 
5. Opportunity: A favorable circumstance for progress and success, arising from changes, trends, or market demand.
6. Affordability: Considering the costs associated with the location, including rent, utilities, and taxes, to ensure financial feasibility.
7. Crime: Prioritizing the safety of customers and staff by selecting a location in a secure neighborhood.
8. Growth Potential: Assessing the growth potential of the surrounding area in terms of population, development plans, and economic indicators.


Ultimately, these categories create a linear objective function. The coefficients will vary depending on what type of business the user is interested in.  PuLP would be implemented to find the optimal datapoint for the Opportunity score. Each block group would then be ranked based on the distance away from the optimal point.  

Opportunity Score = A*[Demographics] + B*[Population Density] + C*[Foot Traffic] - D*[Competition] + E*[Opportunity] + F*[Affordability] - G*[Crime] + H*[Growth Potential]
           

In [26]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection

from pulp import *

import pandas as pd

import psycopg2

from sklearn.preprocessing import StandardScaler

import sys
sys.path.append('../../')
from utils import get_config

sys.path.append('../')
from gis_resources import san_diego_county_zips,  execute_sql, create_where_clause, read_exact_food_biz_categories, read_exact_unhealthy_food_biz_categories

In [27]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

## Query feature layers into dataframes

TODO: Add note that some variables that seems like they should be included were excluded bc they were subsets of existing variables

Values Needed

Demographics: Diversity Index  ([DIVINDX_CY]) 

Population Density: Population per square mile ([POPDENS_CY])

Foot Traffic: Sum the number of retail and other entertainment businesses per square mile (S09_BUS + S26_BUS + S29_BUS/sqm) 

(neg) Competition:  The percentage of food related businesses in the area (S12_BUS + S16_BUS + N13_BUS +N35_BUS/S01_BUS) 

Opportunity:  Current away from home spending rate and the projected 5 year away from home spending growth ((X1130_A / X1002_A) * (X1130FY_A / X1002FY_A))

Affordability: Housing affordability index (HAI_CY)

(neg) Safety: Total Crime Index AGS (CRMCYTOTC) 

Growth Potential: 5 year population growth rate  (TOTPOP_CY/TOTPOP_FY)


Ultimately breaking down the equation as follows: 


Objective Function = A*[DIVINDX_CY] + B*[POPDENS_CY] + C*[S17_BUS + S17_BUS + S26_BUS + S29_BUS]/[SQ Mile] - D*[S12_SALES + S16_BUS + N13_BUS + N37_BUS + N35_BUS]/[S01_BUS] + E*[X1130_A*X1130FY_A]/[X1002_A*X1002FY_A] + F*[INDWHTR_CY + INDRTTR_CY + INDINFO_CY + INDFIN_CY + INDTECH_CY + INDEDUC_CY + INDHLTH_CY + INDARTS_CY] /[sq miles] + G*[HAI_CY] - H*[CRMCYTOTC] + I*[TOTPOP_CY]/[TOTPOP_FY]


In [15]:
needed_cols = ['DIVINDX_CY', 'POPDENS_CY', 'S09_BUS',
               'S26_BUS', 'S29_BUS', 'S12_BUS', 'S16_BUS',
               'N13_BUS', 'N35_BUS', 'S01_BUS', 'X1130_A', 
              'X1130FY_A', 'X1002_A', 'X1002FY_A', 'HAI_CY', 'CRMCYTOTC', 'TOTPOP_CY', 'TOTPOP_FY'] ##TODO fix s17

In [28]:
cs_out_fields = ['fips', 'X1130_A', 'X1130FY_A', 'X1002FY_A', 'x1002_a']

consumer_spending_layer = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a2afc4/FeatureServer/0")

consumer_spending_layer_df = consumer_spending_layer.query(out_fields=cs_out_fields,
                              as_df=True,
                              return_geometry=False)

del consumer_spending_layer_df['FID']

consumer_spending_layer_df.head(3)


,fips,x1002_a,x1002fy_a,x1130_a,x1130fy_a
0,060730100101,6877.52,8470.91,2942.05,3623.67
1,060730100102,8364.19,10237.27,3457.18,4231.38
2,060730100103,7101.43,7930.31,2935.24,3277.84


In [29]:
business_variables_layer = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a633a0/FeatureServer/0")

bus_out_fields = ['fips', 'S09_BUS', 
               'S26_BUS', 'S29_BUS','S12_BUS', 'S16_BUS',
               'N13_BUS', 'N37_BUS', 'N35_BUS', 'S01_BUS']

business_variables_layer_df = business_variables_layer.query(out_fields=bus_out_fields,
                              as_df=True,
                              return_geometry=False)

del business_variables_layer_df['FID']
business_variables_layer_df.head(3)


,fips,n13_bus,n35_bus,n37_bus,s01_bus,s09_bus,s12_bus,s16_bus,s26_bus,s29_bus
0,060730083111,0.0,0.0,0.0,47.0,1.0,0.0,0.0,1.0,1.0
1,060730083112,0.0,0.0,0.0,25.0,1.0,0.0,0.0,0.0,0.0
2,060730083121,0.0,16.0,12.0,101.0,17.0,0.0,12.0,3.0,6.0


In [30]:
demographics_layer_1 = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/aff1b6/FeatureServer/0")

dem_1_out_fields = ['fips', 'DIVINDX_CY', 'POPDENS_CY', 'TOTPOP_CY']

demographics_layer_1_df = demographics_layer_1.query(out_fields=dem_1_out_fields,
                              as_df=True,
                              return_geometry=False)

del demographics_layer_1_df['FID']

demographics_layer_1_df.head(3)


,divindx_cy,fips,popdens_cy,totpop_cy
0,46.0,060730001001,4714.0,1199.0
1,45.6,060730001002,4993.9,1692.0
2,47.5,060730002011,4614.1,902.0


In [32]:
demographics_layer_2 = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/ab8ff7/FeatureServer/0")

dem_2_out_fields = ['fips', 'TOTPOP_FY', 'HAI_CY']

demographics_layer_2_df = demographics_layer_2.query(out_fields=dem_2_out_fields,
                              as_df=True,
                              return_geometry=False)

del demographics_layer_2_df['FID']

demographics_layer_2_df.head(3)


,fips,hai_cy,totpop_fy
0,060730083111,49.0,1821.0
1,060730083112,49.0,1130.0
2,060730083121,48.0,611.0


Read CSV for CRMCYTOTC

In [33]:
crime_df = pd.read_csv('../resources/only_crime_enriched_san_diego_county_block_groups.csv')
crime_df = crime_df[['fips', 'crmcytotc', 'sqmi']]
crime_df['fips'] = "0" + crime_df['fips'].astype(str)

Combine all dataframes into one. 

In [34]:
opportunity_competition_df = consumer_spending_layer_df.merge(business_variables_layer_df, left_on = 'fips', right_on = 'fips')
opportunity_competition_df = opportunity_competition_df.merge(demographics_layer_1_df, left_on = 'fips', right_on = 'fips')
opportunity_competition_df = opportunity_competition_df.merge(demographics_layer_2_df, left_on = 'fips', right_on = 'fips')
opportunity_competition_df = opportunity_competition_df.merge(crime_df, left_on = 'fips', right_on = 'fips')

opportunity_competition_df.head(4)

,fips,x1002_a,x1002fy_a,x1130_a,x1130fy_a,n13_bus,n35_bus,n37_bus,s01_bus,s09_bus,...,s16_bus,s26_bus,s29_bus,divindx_cy,popdens_cy,totpop_cy,hai_cy,totpop_fy,crmcytotc,sqmi
0,060730100101,6877.52,8470.91,2942.05,3623.67,0.0,0.0,0.0,11.0,2.0,...,0.0,0.0,1.0,78.7,15154.6,1690.0,81.0,1654.0,52.0,0.11
1,060730100102,8364.19,10237.27,3457.18,4231.38,2.0,3.0,3.0,25.0,8.0,...,2.0,1.0,1.0,80.8,9018.8,2185.0,53.0,2136.0,82.0,0.24
2,060730100103,7101.43,7930.31,2935.24,3277.84,1.0,3.0,3.0,29.0,13.0,...,3.0,1.0,0.0,84.8,15980.3,1565.0,201.0,1530.0,104.0,0.10
3,060730100111,10566.02,12361.14,4519.91,5287.83,1.0,1.0,1.0,16.0,2.0,...,1.0,2.0,2.0,84.2,7335.9,1485.0,79.0,1496.0,146.0,0.20


Create sub scores

In [39]:
opportunity_competition_df['demographics'] = opportunity_competition_df['divindx_cy']

opportunity_competition_df['population_density'] = opportunity_competition_df['popdens_cy']

opportunity_competition_df['foot_traffic'] = (opportunity_competition_df['s09_bus'] +  
                                             opportunity_competition_df['s26_bus'] + 
                                             opportunity_competition_df['s29_bus'])/opportunity_competition_df['sqmi']
    
opportunity_competition_df['competition'] = (opportunity_competition_df['s12_bus'] + 
                                            opportunity_competition_df['s16_bus'] + 
                                            opportunity_competition_df['n13_bus'] + 
                                            opportunity_competition_df['n35_bus'])/opportunity_competition_df['s01_bus']

opportunity_competition_df['opportunity'] = (opportunity_competition_df['x1130_a'] *
                                            opportunity_competition_df['x1130fy_a']) / (opportunity_competition_df['x1002_a'] *
                                            opportunity_competition_df['x1002fy_a'])  

opportunity_competition_df['affordability'] = opportunity_competition_df['hai_cy']

opportunity_competition_df['crime'] = opportunity_competition_df['crmcytotc']

opportunity_competition_df['growth_potential'] = opportunity_competition_df['totpop_cy'] / opportunity_competition_df['totpop_fy']


In [40]:
score_variables = opportunity_competition_df[['fips', 'demographics', 'population_density', 'foot_traffic', 'competition',
                                             'opportunity', 'affordability', 'crime',
                                             'growth_potential']]

score_variables['opportunity'] = score_variables['opportunity'].astype(float)

/var/folders/q1/22xbrl4x06zbl30hwjxzsfm80000gr/T/ipykernel_86442/2427797150.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score_variables['opportunity'] = score_variables['opportunity'].astype(float)


Normalize the data per col

In [41]:
scaler = StandardScaler()
score_variables_standardized = score_variables.copy().set_index('fips')
score_variables_standardized[score_variables_standardized.columns] = scaler.fit_transform(score_variables_standardized)
score_variables_standardized

##cap values at 4 
def truncate_std(std, boundary = 4):
    if std > boundary:
        return boundary
    elif std < -boundary:
        return -boundary
    else:
        return std
    
for col in score_variables_standardized.columns:
    score_variables_standardized[col] = score_variables_standardized[col].apply(lambda x: truncate_std(x))


# Use PuLP to calculate the score 


In [43]:
problem = LpProblem("Opportunity Problem", LpMaximize)

demographics = LpVariable("demographics", score_variables_standardized['demographics'].min(), score_variables_standardized['demographics'].max())
population_density = LpVariable("population_density", score_variables_standardized['population_density'].min(), score_variables_standardized['population_density'].max())
foot_traffic = LpVariable("foot_traffic", score_variables_standardized['foot_traffic'].min(), score_variables_standardized['foot_traffic'].max())
competition = LpVariable("competition", score_variables_standardized['competition'].min(), score_variables_standardized['competition'].max())
opportunity = LpVariable("opportunity", score_variables_standardized['opportunity'].min(), score_variables_standardized['opportunity'].max())
affordability = LpVariable("affordability", score_variables_standardized['affordability'].min(), score_variables_standardized['affordability'].max())
crime = LpVariable("crime", score_variables_standardized['crime'].min(), score_variables_standardized['crime'].max())
growth_potential = LpVariable("growth_potential", score_variables_standardized['growth_potential'].min(), score_variables_standardized['growth_potential'].max())


problem += (10*demographics) + (10*population_density) + (10*foot_traffic) - (10*competition) + (20*opportunity) + (20*affordability) - (10*crime) + (10*growth_potential)
        

problem.writeLP("OpportunityCompetition.lp")
problem.solve()

print("Status:", LpStatus[problem.status])

# Each of the variables is printed with it's resolved optimum value
for v in problem.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Most optimal sum = ", value(problem.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jessica.allen/PycharmProjects/capstone_again/venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/q1/22xbrl4x06zbl30hwjxzsfm80000gr/T/34cf7decac47417da72d0ea05d6a09a1-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/q1/22xbrl4x06zbl30hwjxzsfm80000gr/T/34cf7decac47417da72d0ea05d6a09a1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5 COLUMNS
At line 14 RHS
At line 15 BOUNDS
At line 32 ENDATA
Problem MODEL has 0 rows, 8 columns and 0 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Empty problem - 0 rows, 8 columns and 0 elements
Optimal - objective value 267.30182
Optimal objective 267.3018166 - 0 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Status: Optimal
affo

In [45]:
score_variables_standardized['score'] = (10*score_variables_standardized['demographics']) + (10*score_variables_standardized['population_density']) + (10*score_variables_standardized['foot_traffic']) - (10*score_variables_standardized['competition']) + (20*score_variables_standardized['opportunity']) + (20*score_variables_standardized['affordability']) - (10*score_variables_standardized['crime']) + (10*score_variables_standardized['growth_potential'])
score_variables_standardized['distance_from_optimal_score'] = value(problem.objective) - score_variables_standardized['score']

In [46]:
score_variables_standardized.sort_values('distance_from_optimal_score').head(10)

,demographics,population_density,foot_traffic,competition,opportunity,affordability,crime,growth_potential,score,distance_from_optimal_score
fips,,,,,,,,,,
060730157013,1.067218,3.491881,-0.041536,-0.411435,-0.267071,4.000000,-0.412021,0.430551,132.374273,134.927543
060730009022,0.260763,2.015658,2.037898,-0.713357,1.584182,2.983194,0.600427,-1.288506,122.734959,144.566857
060730204041,1.074755,0.613797,-0.422875,-0.668843,0.315151,4.000000,-0.918245,0.493919,119.769875,147.531942
060730157051,0.396429,2.067344,0.267690,-0.107965,-0.000777,4.000000,-0.719371,0.406237,119.634806,147.667010
060730101101,0.991849,-0.064068,-0.164459,0.804171,0.952937,4.000000,-0.176988,0.550068,105.920819,161.380997
060730157012,1.029533,0.678063,-0.016716,-0.583861,-0.267037,4.000000,-0.249306,0.538238,105.282129,162.019687
060730157061,0.728055,1.675290,0.119799,-0.225413,-0.536232,4.000000,-0.321624,0.515627,105.133443,162.168373
060730137022,1.127514,0.523007,-0.182188,-0.980442,-0.267101,4.000000,0.039965,0.477595,103.522019,163.779797
060730165041,1.089829,0.825740,-0.273633,0.421754,-0.267079,4.000000,-0.918245,0.327494,99.317637,167.984179


In [47]:
final_df = score_variables_standardized.reset_index()
final_df = final_df[['fips', 'distance_from_optimal_score']].dropna()

In [32]:
final_df.sort_values('distance_from_optimal_score')

,fips,distance_from_optimal_score
468,060730008004,120.879370
889,060730009022,134.488452
1185,060730083634,134.926905
1120,060730051022,135.098954
1133,060730053022,139.567143
...,...,...
2022,060730209044,374.421073
1507,060730209021,377.357799
1913,060730209043,378.040160
1834,060730183012,383.449495
